In [ ]:

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
setHadoopConfigWithCredentials_d3bd5b94a9334de59a55a7fed2bedeaa <- function(name) {
    # This function sets the Hadoop configuration so it is possible to
    # access data from Bluemix Object Storage using Spark

    prefix = paste("fs.swift.service" , name, sep =".")
    hConf = SparkR:::callJMethod(sc, "hadoopConfiguration")
    SparkR:::callJMethod(hConf, "set", paste(prefix, "auth.url", sep='.'), paste("https://identity.open.softlayer.com","/v3/auth/tokens",sep=""))
    SparkR:::callJMethod(hConf, "set", paste(prefix, "auth.endpoint.prefix", sep='.'), "endpoints")
    SparkR:::callJMethod(hConf, "set", paste(prefix, "tenant", sep='.'), "6aaf54352357483486ee2d4981f8ef15")
    SparkR:::callJMethod(hConf, "set", paste(prefix, "username", sep='.'), "c0eebedc019f4413be3f3d656821b35f")
    SparkR:::callJMethod(hConf, "set", paste(prefix, "password", sep='.'), "ji[T[l.(7D&gld*5")
    SparkR:::callJMethod(hConf, "set", paste(prefix, "region", sep='.'), "dallas")
    invisible(SparkR:::callJMethod(hConf, "setBoolean", paste(prefix, "public", sep='.'), FALSE))
}

name <- "keystone"
setHadoopConfigWithCredentials_d3bd5b94a9334de59a55a7fed2bedeaa(name)

invisible(sparkR.session(appName = "test SparkSession R"))




In [ ]:
df.data.1 <- read.json(paste("swift://", "coursera", "." , name,"/", "bearing1_1_acc_transformed4.json", sep=""), 
                       source = "org.apache.spark.sql.execution.datasources.csv.CSVFileFormat", header = "true")
head(df.data.1)

In [ ]:
n = nrow(df.data.1)
n

In [ ]:
createOrReplaceTempView(df.data.1,"data")
df_sample = sql("select * from data where rand() <= .1 order by ts asc")

In [ ]:
n = nrow(df_sample)
n

In [ ]:
df_sample_rdf = collect(df_sample)

In [ ]:
colnames(df_sample_rdf)

In [ ]:
attach(df_sample_rdf)
plot(ts,hacc, type="o", col="blue")
detach(df_sample_rdf)

In [ ]:
attach(df_sample_rdf)
plot(ts,vacc, type="o", col="blue")
detach(df_sample_rdf)

In [ ]:
df_grouped = sql("
    select cluster,
    mean(hacc) as mhacc,
    mean(vacc) as mvacc,
    STDDEV_POP(hacc) as sdhacc,
    STDDEV_POP(vacc) as sdvacc 
    from data 
    group by cluster 
    order by cluster asc")

In [ ]:
df_grouped_local = collect(df_grouped)

In [ ]:
df_grouped_local

In [ ]:
nrow(df_grouped_local)

In [ ]:
attach(df_grouped_local)
plot(cluster,sdhacc)
detach(df_grouped_local)

In [ ]:
attach(df_grouped_local)
plot(cluster,sdvacc)
detach(df_grouped_local)

In [ ]:
attach(df_grouped_local)
plot(cluster,mhacc)
detach(df_grouped_local)

In [ ]:
attach(df_grouped_local)
plot(cluster,mvacc)
detach(df_grouped_local)

In [ ]:
install.packages("wavelets")

In [ ]:
library(wavelets)

In [ ]:
attach(df_sample_rdf)
wt = dwt(vacc, filter="haar", boundary="periodic")
detach(df_sample_rdf)

In [ ]:
head(unlist(c(wt@W,wt@V[[wt@level]])))